# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jinjiang,24.8198,118.5741,16.68,74,100,6.33,CN,1706848121
1,1,maceio,-9.6658,-35.7353,25.69,94,75,1.54,BR,1706848121
2,2,port mathurin,-19.6833,63.4167,27.48,82,27,11.30,MU,1706848121
3,3,chegdomyn,51.1167,133.0167,-22.57,81,17,0.94,RU,1706848121
4,4,dudinka,69.4058,86.1778,-13.54,91,100,11.59,RU,1706848121


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng',
                                      y='Lat', 
                                      color='City', 
                                      geo=True,
                                      tiles="OSM",  
                                      frame_width=700,
                                      frame_height=500,
                                      size='Humidity',
                                      scale=0.9,
                                      hover_cols=['City','Country','Humidity']
                                      )
                                     


# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp_criteria = (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30)
humidity_criteria = city_data_df['Humidity'] > 50
cloudiness_criteria = city_data_df['Cloudiness'] < 10
wind_speed_criteria = city_data_df['Wind Speed'] < 10

filtered_df = city_data_df[max_temp_criteria & humidity_criteria & cloudiness_criteria & wind_speed_criteria]

ideal_conditions_df = filtered_df.copy()


# Drop any rows with null values
ideal_conditions_df.dropna(inplace=True)

# Display sample data
ideal_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,cabo san lucas,22.8909,-109.9124,22.51,76,0,1.79,MX,1706848133
155,155,petrolina,-9.3986,-40.5008,26.03,73,0,2.06,BR,1706847926
181,181,nelson bay,-32.7167,152.1500,29.49,73,1,2.68,AU,1706848143
319,319,mangrol,21.1167,70.1167,22.77,65,6,2.58,IN,1706848160
356,356,kudahuvadhoo,2.6708,72.8944,27.87,77,9,5.57,MV,1706848166
375,375,mar del plata,-38.0023,-57.5575,23.30,81,0,3.13,AR,1706847956
386,386,mumbai,19.0144,72.8479,25.99,53,0,2.06,IN,1706847639
438,438,tapes,-30.6733,-51.3958,21.81,84,0,4.56,BR,1706848179
472,472,imeko,7.4500,2.8500,24.72,89,7,1.33,NG,1706848183
484,484,pisco,-13.7000,-76.2167,24.03,78,0,5.14,PE,1706848185


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
98,cabo san lucas,MX,22.8909,-109.9124,76,
155,petrolina,BR,-9.3986,-40.5008,73,
181,nelson bay,AU,-32.7167,152.1500,73,
319,mangrol,IN,21.1167,70.1167,65,
356,kudahuvadhoo,MV,2.6708,72.8944,77,
375,mar del plata,AR,-38.0023,-57.5575,81,
386,mumbai,IN,19.0144,72.8479,53,
438,tapes,BR,-30.6733,-51.3958,84,
472,imeko,NG,7.4500,2.8500,89,
484,pisco,PE,-13.7000,-76.2167,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories= "accommodation.hotel"
apiKey   = geoapify_key
limit=10

params = {
    "categories": categories,
    "radius": radius,
    "limit": limit, 
    "apiKey": geoapify_key
}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
petrolina - nearest hotel: Neuman
nelson bay - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel found
kudahuvadhoo - nearest hotel: Hotel Riu Palace Maldivas
mar del plata - nearest hotel: Gran hotel Mónaco
mumbai - nearest hotel: Hotel City Point
tapes - nearest hotel: No hotel found
imeko - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
cidreira - nearest hotel: Hotel Castelo


,City,Country,Lat,Lng,Humidity,Hotel Name
98,cabo san lucas,MX,22.8909,-109.9124,76,Comfort Rooms
155,petrolina,BR,-9.3986,-40.5008,73,Neuman
181,nelson bay,AU,-32.7167,152.1500,73,No hotel found
319,mangrol,IN,21.1167,70.1167,65,No hotel found
356,kudahuvadhoo,MV,2.6708,72.8944,77,Hotel Riu Palace Maldivas
375,mar del plata,AR,-38.0023,-57.5575,81,Gran hotel Mónaco
386,mumbai,IN,19.0144,72.8479,53,Hotel City Point
438,tapes,BR,-30.6733,-51.3958,84,No hotel found
472,imeko,NG,7.4500,2.8500,89,No hotel found
484,pisco,PE,-13.7000,-76.2167,78,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
map_plot = hotel_df.hvplot.points(x='Lng',
                                  y='Lat', 
                                  color='City', 
                                  geo=True,
                                  tiles="OSM",  
                                  frame_width=700,
                                  frame_height=500,
                                  size='Humidity',
                                  scale= 2,
                                  hover_cols=['City','Country','Hotel Name']
                                  )
                                     



# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)